# **Overview**

This system is equipped with a stationary camera strategically positioned to observe the designated area where aircraft are prepared for departure. The primary objective is to ensure the absence of any vehicles within the stand boundary before the aircraft arrives. In the event of detecting any vehicles within this zone, the system is designed to promptly notify the ground handler for immediate action

# Libraries

In [1]:
import cv2
import numpy as np
from sklearn.metrics import recall_score, precision_score

from ultralytics import YOLO

import json
import os

from src.predictor import Predictor
from src.utils import convert_intervals_to_list

# **Data**

In [2]:
with open('data/polygons.json') as f:
    polygons = json.load(f)

In [3]:
with open('data/time_intervals.json') as f:
    time_intervals = json.load(f)

In [4]:
polygons

{'video_0.mp4': [[536, 573],
  [873, 562],
  [824, 422],
  [933, 420],
  [877, 303],
  [404, 318],
  [401, 437],
  [542, 437],
  [542, 438]],
 'video_1.mp4': [[536, 573],
  [873, 562],
  [824, 422],
  [933, 420],
  [877, 303],
  [404, 318],
  [401, 437],
  [542, 437],
  [542, 438]],
 'video_2.mp4': [[488, 701],
  [915, 673],
  [700, 429],
  [868, 417],
  [799, 301],
  [112, 321],
  [116, 466],
  [439, 446]],
 'video_3.mp4': [[256, 718],
  [370, 509],
  [0, 461],
  [312, 212],
  [1069, 203],
  [1279, 297],
  [1277, 718]],
 'video_4.mp4': [[256, 718],
  [370, 509],
  [0, 461],
  [312, 212],
  [1069, 203],
  [1279, 297],
  [1277, 718]],
 'video_5.mp4': [[515, 482],
  [282, 486],
  [321, 444],
  [240, 427],
  [363, 371],
  [715, 367],
  [896, 406],
  [869, 419],
  [948, 442],
  [796, 460],
  [1115, 597],
  [538, 595]],
 'video_6.mp4': [[1181, 698],
  [0, 560],
  [0, 377],
  [868, 366],
  [1279, 430],
  [1279, 594]],
 'video_7.mp4': [[604, 718],
  [0, 552],
  [0, 461],
  [132, 400],
  [837,

In [5]:
time_intervals

{'video_0.mp4': [],
 'video_1.mp4': [[4, 12]],
 'video_2.mp4': [[40, 43], [57, 60]],
 'video_3.mp4': [[43, 50]],
 'video_4.mp4': [[19, 52]],
 'video_5.mp4': [[14, 19], [34, 38], [73, 95]],
 'video_6.mp4': [[2, 109], [123, 129]],
 'video_7.mp4': [[0, 128]],
 'video_8.mp4': [],
 'video_9.mp4': [],
 'video_10.mp4': [[0, 2]],
 'video_11.mp4': [[0, 21]],
 'video_13.mp4': [[29, 78], [134, 158]],
 'video_14.mp4': [[24, 24], [37, 37], [63, 64]],
 'video_15.mp4': [[10, 29]],
 'video_16.mp4': [[8, 23]],
 'video_17.mp4': [[43, 50]],
 'video_18.mp4': [[0, 87]],
 'video_19.mp4': [[6, 8], [36, 58], [147, 177]]}

# **Metrics**

Precision and recall are commonly used metrics in binary classification tasks, and they are particularly relevant for evaluating the performance of a system designed to monitor and notify the presence of vehicles on the aircraft stand. Let's delve into the reasons for choosing precision and recall and how they align with the task:

1. **Precision:**
   - Precision is the ratio of true positive predictions to the total number of positive predictions (true positives + false positives).
   - In the context of the aircraft stand monitoring system, precision is crucial because it measures the accuracy of the system in identifying and alerting about vehicles on the stand. A high precision value indicates that when the system notifies the ground handler, it is likely to be correct and that there are indeed vehicles present.

2. **Recall:**
   - Recall, also known as sensitivity or true positive rate, is the ratio of true positive predictions to the total number of actual positive instances (true positives + false negatives).
   - Recall is significant in this task as it gauges the system's ability to capture and identify all instances of vehicles on the stand. A high recall value implies that the system is effective in minimizing false negatives, ensuring that it reliably detects and notifies the ground handler whenever a vehicle is present.

In the context of the aircraft stand monitoring system:

- **False Positives (FP):** Predicting a vehicle is present when it is not.
- **False Negatives (FN):** Failing to predict a vehicle that is present.

Here's how precision and recall relate to the task:

- **High Precision:** Indicates a low rate of false positives. The ground handler can trust the system's alerts, as they are likely accurate.
  
- **High Recall:** Signifies a low rate of false negatives. The system consistently detects vehicles on the stand, reducing the risk of missing any instances.

Balancing precision and recall is often essential, as there is typically a trade-off between the two. Depending on the system's requirements and priorities, one may need to adjust the model or algorithm parameters to achieve the desired balance between minimizing false positives and false negatives.

# **Approach**

In [15]:
model = YOLO('models/yolov8s.pt')

100%|█████████████████████████████████████████████████████████████| 21.5M/21.5M [00:01<00:00, 15.0MB/s]


In [11]:
predictor = Predictor(model=model)

In [12]:
metrics = {}

In [ ]:
for video_name in os.listdir('data/videos/')[1:]:
    print(video_name)
    prediction, predicted_intervals = predictor.predict_vehicle_entity(f'data/videos/{video_name}', polygons[video_name])

    target_intervals = time_intervals[video_name]
    target = convert_intervals_to_list(target_intervals, len(prediction))

    recall = recall_score(target, prediction, zero_division=1)
    precision = precision_score(target, prediction, zero_devision=1)

    metric = {'precision': precision, 'recall': recall}
    metrics[video_name] = metric

Below are the evaluation metrics for the aircraft stand monitoring system applied to different video files. The metrics include precision and recall values for each video, providing insights into the system's performance in identifying and notifying the presence of vehicles on the aircraft stand

- **video_11.mp4**
Precision: 1.0
Recall: 1.0
- **video_13.mp4**
Precision: 0.4237
Recall: 1.0
- **video_16.mp4**
Precision: 0.5333
Recall: 1.0
- **video_17.mp4**
Precision: 0.1569
Recall: 1.0
- **video_8.mp4**
Precision: 0.0
Recall: 0.0
- **video_15.mp4**
Precision: 1.0
Recall: 0.9
- **video_14.mp4**
Precision: 0.0615
Recall: 1.0
- **video_9.mp4**
Precision: 0.0
Recall: 0.0
- **video_4.mp4**
Precision: 0.5152
Recall: 1.0
- **video_19.mp4**
Precision: 0.3709
Recall: 0.9825
- **video_18.mp4**
Precision: 1.0
Recall: 1.0
- **video_5.mp4**
Precision: 0.2378
Recall: 1.0
- **video_7.mp4**
Precision: 1.0
Recall: 0.7597
- **video_6.mp4**
Precision: 0.8906
Recall: 0.9913
- **video_2.mp4**
Precision: 0.8571
Recall: 0.75
- **video_3.mp4**
Precision: 0.1455
Recall: 1.0
- **video_1.mp4**
Precision: 1.0
Recall: 0.3333
- **video_0.mp4**
Precision: 0.0
Recall: 0.0

These metrics offer a detailed assessment of the system's ability to accurately identify the presence of vehicles on the stand. While the results indicate notable performance in certain cases, it's important to note that due to resource limitations, model improvements, such as employing a larger model, are currently unfeasible. Ongoing efforts will focus on optimizing within the existing resource constraints for enhanced precision and recall across all scenarios

In [14]:
metrics

{'video_11.mp4': {'precision': 1.0, 'recall': 1.0},
 'video_13.mp4': {'precision': 0.423728813559322, 'recall': 1.0},
 'video_16.mp4': {'precision': 0.5333333333333333, 'recall': 1.0},
 'video_17.mp4': {'precision': 0.1568627450980392, 'recall': 1.0},
 'video_8.mp4': {'precision': 0.0, 'recall': 0.0},
 'video_15.mp4': {'precision': 1.0, 'recall': 0.9},
 'video_14.mp4': {'precision': 0.06153846153846154, 'recall': 1.0},
 'video_9.mp4': {'precision': 0.0, 'recall': 0.0},
 'video_4.mp4': {'precision': 0.5151515151515151, 'recall': 1.0},
 'video_19.mp4': {'precision': 0.3708609271523179,
  'recall': 0.9824561403508771},
 'video_18.mp4': {'precision': 1.0, 'recall': 1.0},
 'video_5.mp4': {'precision': 0.23776223776223776, 'recall': 1.0},
 'video_7.mp4': {'precision': 1.0, 'recall': 0.7596899224806202},
 'video_6.mp4': {'precision': 0.890625, 'recall': 0.991304347826087},
 'video_2.mp4': {'precision': 0.8571428571428571, 'recall': 0.75},
 'video_3.mp4': {'precision': 0.14545454545454545, 're